In [ ]:
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import numpy as np
import glob
import sys
sys.path.append('/Users/david/Dropbox/PhD/Scripts/Spatial analyses')
import psycopg2
from pathlib import Path
from shapely.geometry import Point, Polygon
import geoplot
import geoplot.crs as gcrs
import datashader as ds, colorcet as cc
# import holoviews as hv
# hv.extension("bokeh")
# from holoviews.element.tiles import EsriImagery
import rasterio
from rasterio.mask import mask
from rasterio.plot import show
import fiona
import seaborn as sns
from datashader.utils import export_image
# from holoviews.operation.datashader import datashade
import matplotlib.pyplot as plt
import libpysal as lps
from scipy.spatial import cKDTree
from libpysal.weights.distance import get_points_array
from esda import fdr
import contextily as ctx
import pyspace
from importlib import reload
plt.rc('font', family='Helvetica')

In [ ]:
def classy_map_ch(df, col, cmap, title, _reversed = True):
    filename = title+'.png'
    if not os.path.isfile(result_folder/'Maps features'/filename):
        if _reversed:
            cmap += '_r'
        ax = df.to_crs(21781).plot(col, markersize=0.05, linewidth = 0.1, cmap = cmap, legend = True,figsize = (8, 8), legend_kwds = {'shrink':0.5})
        lakes.plot(color = 'lightblue', ax=ax)
        cantons_ch.geometry.boundary.plot(ax=ax,edgecolor='k', color=None, linewidth=0.1)
        plt.imshow(out_image.squeeze(),extent=ch_extent, cmap='Greys_r', alpha=0.4)
        ax.set_axis_off()
        ax.set_title(title, fontsize=15, color= 'grey')
        plt.savefig(result_folder/'Maps features'/filename, dpi=600, bbox_inches='tight')
        return ax
    else:
        print('Map already generated')

In [ ]:
def show_values(axs, orient="v",digits = 2, fontsize = 8, space=.05):
    def _single(ax):
        if orient == "v":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() / 2
                _y = p.get_y() + p.get_height() + (p.get_height()*0.02)
                value = '{:.{}f}'.format(p.get_height(), digits)
                ax.text(_x, _y, value,size = fontsize, ha="center") 
        elif orient == "h":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() + float(space)
                _y = p.get_y() + p.get_height() - (p.get_height()*0.5)
                value = '{:.{}f}'.format(p.get_width(), digits)
                ax.text(_x, _y, value,size = fontsize, ha="left")

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _single(ax, digits)
    else:
        _single(axs)
def optimize_df(df):
    """
    Convert each column of a pandas DataFrame to the datatype that takes the lowest memory.

    Parameters:
    -----------
    df : pandas DataFrame
        The input DataFrame to convert.

    Returns:
    --------
    pandas DataFrame
        The converted DataFrame with lowest memory datatypes for each column.
    """

    # First, convert all object columns to category type
    obj_cols = df.select_dtypes(include=['object']).columns
    df[obj_cols] = df[obj_cols].astype('category')

    # Next, loop through all numeric columns and downcast the data types
    for col in df.select_dtypes(include=['int', 'float']).columns:
        col_type = df[col].dtype
        if str(col_type)[:3] == 'int':
            # Use smallest integer type possible
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.int64)
#         else:
#             # Use smallest float type possible ! Bug 'halffloat' not supported by Arrow ! -> Commenting out
#             c_min = df[col].min()
#             c_max = df[col].max()
#             if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                 df[col] = df[col].astype('float32')
#             else:
#                 df[col] = df[col].astype('float64')
    
    return df

In [ ]:
data_folder  = Path('../data/')
result_folder = Path('../output')

In [ ]:
## Insurance claims

# Linkage
df_paires_lamal_lca = pd.read_csv(data_folder/'max_probs_w_zipcode_pour_david_w_uuid.csv')

dict_lamal_to_uuid = df_paires_lamal_lca.set_index('id_lamal')['uuid'].to_dict()
dict_lca_to_uuid = df_paires_lamal_lca.set_index('id_lca')['uuid'].to_dict()

dict_lamal_to_lca = df_paires_lamal_lca.set_index('id_lamal')['id_lca'].to_dict()
dict_lca_to_lamal = df_paires_lamal_lca.set_index('id_lca')['id_lamal'].to_dict()

df_aos_address = optimize_df(pd.read_parquet(data_folder/'raw'/'GM'/'Full'/'Compressed files'/'df_aos_address.parquet.gzip'))
df_multiple_address_aos = optimize_df(pd.read_parquet(data_folder/'raw'/'GM'/'Full'/'Compressed files'/'df_multiple_address_aos.parquet.gzip'))
df_multiple_address_lca = optimize_df(pd.read_parquet(data_folder/'raw'/'GM'/'Full'/'Compressed files'/'df_multiple_address_lca.parquet.gzip'))
df_remaining_multiple_lca = optimize_df(pd.read_parquet(data_folder/'raw'/'GM'/'Full'/'Compressed files'/'df_remaining_multiple_lca.parquet.gzip'))
##
df_couverture_aos = optimize_df(pd.read_parquet(data_folder/'raw'/'GM'/'Full'/'Compressed files'/'df_couverture_aos.parquet.gzip'))
df_flag_aos = optimize_df(pd.read_parquet(data_folder/'raw'/'GM'/'Full'/'Compressed files'/'df_flag_aos.parquet.gzip'))
df_drug_aos = optimize_df(pd.read_parquet(data_folder/'raw'/'GM'/'Full'/'Compressed files'/'df_drug_aos.parquet.gzip'))
df_couverture_lca = optimize_df(pd.read_parquet(data_folder/'raw'/'GM'/'Full'/'Compressed files'/'df_couverture_lca.parquet.gzip'))
df_prestation_aos = optimize_df(pd.read_parquet(data_folder/'raw'/'GM'/'Full'/'Compressed files'/'df_prestation_aos.parquet.gzip'))
df_prestation_lca = optimize_df(pd.read_parquet(data_folder/'raw'/'GM'/'Full'/'Compressed files'/'df_prestation_lca.parquet.gzip'))

In [ ]:
df_prestation_aos_cam = pd.read_csv(data_folder/'raw'/'GM'/'Full'/'Compressed files'/'Santeintegra_TARMED_080523.csv', sep=';')

In [ ]:
# Mapping between TARMED codes and their associated category
dict_tarmed_cam = {'0.1710':'Acupuncture',
                   '0.1720':'Acupuncture',
                   '0.1730':'Acupuncture',
                   '0.1735':'Acupuncture',
                   '0.1740':'Neural therapy',
                   '0.1750':'Neural therapy',
                   '0.1760':'Neural therapy',
                   '0.1770':'Homeopathy',
                   '0.1780':'Homeopathy',
                   '0.1790':'Homeopathy',
                   '0.1800':'Homeopathy',
                   '0.1800':'Homeopathy',
                   '0.1810':'Traditional Chinese medicine',
                   '0.1820':'Traditional Chinese medicine',
                   '0.1830':'Traditional Chinese medicine',
                   '0.1840':'Anthroposophic medicine',
                   '0.1850':'Anthroposophic medicine',
                   '0.1860':'Anthroposophic medicine',
                   '0.1870':'Phytotherapy',
                   '0.1871':'Phytotherapy',
                   '0.1872':'Phytotherapy',
                   '0.1880':'Phone consultation',
                   '0.1890':'Phone consultation',
                   '0.1895':'Phone consultation',
                   '0.1896':'Phone consultation',
                   '0.1900':'Phone consultation',

                  }

In [ ]:
df_prestation_aos_cam['CDPOSITION_categories'] = df_prestation_aos_cam['CDPOSITION'].map(df_prestation_aos_cam['CDPOSITION'])

In [ ]:
# Add a UUID so that we have a single unique ID instead of pairs of ID_LAMAL-ID_LCA
df_prestation_lca['uuid'] = df_prestation_lca['ID_LCA'].map(dict_lca_to_uuid)
df_prestation_aos['uuid'] = df_prestation_aos['ID_LAMAL'].map(dict_lamal_to_uuid)
df_prestation_aos_cam['uuid'] = df_prestation_aos_cam['ID_LAMAL'].map(dict_lamal_to_uuid)

df_couverture_lca['uuid'] = df_couverture_lca['ID_LCA'].map(dict_lca_to_uuid)
df_couverture_aos['uuid'] = df_couverture_aos['ID_LAMAL'].map(dict_lamal_to_uuid)

df_drug_aos['uuid'] = df_drug_aos['ID_LAMAL'].map(dict_lamal_to_uuid)
df_flag_aos['uuid'] = df_flag_aos['ID_LAMAL'].map(dict_lamal_to_uuid)

In [ ]:
# Convert to GeoDataFrames
df_aos_address = gpd.GeoDataFrame(df_aos_address, crs = 4326, geometry = gpd.points_from_xy(df_aos_address['lon_masked'], df_aos_address['lat_masked']))
df_multiple_address_aos = gpd.GeoDataFrame(df_multiple_address_aos, crs = 4326, geometry = gpd.points_from_xy(df_multiple_address_aos['lon_masked'], df_multiple_address_aos['lat_masked']))
df_multiple_address_lca = gpd.GeoDataFrame(df_multiple_address_lca, crs = 4326, geometry = gpd.points_from_xy(df_multiple_address_lca['lon_masked'], df_multiple_address_lca['lat_masked']))

In [ ]:
# Convert the date strings to datetime objects
df_multiple_address_aos['date'] = pd.to_datetime(df_multiple_address_aos['MIN_of_Date_adress'], format='%d%b%y')
# Extract the month and year as separate columns
df_multiple_address_aos['month'] = df_multiple_address_aos['date'].dt.month
df_multiple_address_aos['year'] = df_multiple_address_aos['date'].dt.year

In [ ]:
# Convert the date strings to datetime objects
df_multiple_address_lca['date'] = pd.to_datetime(df_multiple_address_lca['MIN_of_Date_adress'], format='%d%b%y')
# Extract the month and year as separate columns
df_multiple_address_lca['month'] = df_multiple_address_lca['date'].dt.month
df_multiple_address_lca['year'] = df_multiple_address_lca['date'].dt.year

In [ ]:
def process_date(df, year_col='ANNEE_TRAITEMENT', month_col='MOIS_TRAITEMENT'):
    df['treatmentdate'] = pd.to_datetime(df[year_col].astype('string') + '-' + df[month_col].astype('string'))
    df['treatmentmonth'] = df['treatmentdate'].dt.strftime('%Y-%m')
    df['treatment_Q'] = df['treatmentdate'].dt.to_period("Q")
    return df

df_list = [df_prestation_aos, df_prestation_lca, df_prestation_aos_cam, df_drug_aos]
for df in df_list:
    df = process_date(df)

In [ ]:
# Optimize dfs
df_prestation_aos = optimize_df(df_prestation_aos)
df_prestation_lca = optimize_df(df_prestation_lca)
df_prestation_aos_cam = optimize_df(df_prestation_aos_cam)
df_drug_aos = optimize_df(df_drug_aos)
df_couverture_aos = optimize_df(df_couverture_aos)
df_couverture_lca = optimize_df(df_couverture_lca)
df_flag_aos = optimize_df(df_flag_aos)

In [ ]:
# Get the ID not present in the respective datasets
df_prestation_aos['ID_LCA'] = df_prestation_aos['ID_LAMAL'].map(dict_lamal_to_lca)
df_prestation_lca['ID_LAMAL'] = df_prestation_lca['ID_LCA'].map(dict_lca_to_lamal)

In [ ]:
dict_cantons = {'AG':'Aargau',
                'AI':'Appenzell Innerrhoden',
                'AR':"Appenzell Ausserrhoden",
                'BE':'Bern',
                'BL':'Basel-Landschaft',
                'BS':'Basel-Stadt',
                'FR':'Fribourg',
                'GE':'Genève',
                'GL':'Glarus',
                'GR':'Graubünden',
                'JU':"Jura",
                'LU':'Luzern',
                'NE':'Neuchâtel',
                'NW':'Nidwalden',
                'OW':'Obwalden',
                'SH':'Schaffhausen',
                'SZ':'Schwyz',
                'SO':'Solothurn',
                'SG':'St. Gallen',
                'TG':'Thurgau',
                'TI':'Ticino',
                'UR':'Uri',
                'VS':'Valais',
                'VD':'Vaud',
                'ZG':'Zug',
                'ZH':'Zürich'}

## Update : New address data sent by Christophe on January 27th, 2023    

In [ ]:
df_aos_address_updated = optimize_df(pd.read_parquet(data_folder/'raw'/'GM'/'Full'/'Compressed files'/'df_aos_address_updated.parquet.gzip'))
df_multiple_address_aos_updated = optimize_df(pd.read_parquet(data_folder/'raw'/'GM'/'Full'/'Compressed files'/'df_multiple_address_aos_updated.parquet.gzip'))
df_multiple_address_lca_updated = optimize_df(pd.read_parquet(data_folder/'raw'/'GM'/'Full'/'Compressed files'/'df_multiple_address_lca_updated.parquet.gzip'))
df_remaining_multiple_lca_updated = optimize_df(pd.read_parquet(data_folder/'raw'/'GM'/'Full'/'Compressed files'/'df_remaining_multiple_lca_updated.parquet.gzip'))
df_remaining_multiple_aos_updated = optimize_df(pd.read_parquet(data_folder/'raw'/'GM'/'Full'/'Compressed files'/'df_remaining_multiple_aos_updated.parquet.gzip'))

In [ ]:
df_multiple_address_aos_updated['date'] = pd.to_datetime(df_multiple_address_aos_updated['MIN_of_Date_adress'], format='%d%b%y')
df_multiple_address_lca_updated['date'] = pd.to_datetime(df_multiple_address_lca_updated['MIN_of_Date_adress'], format='%d%b%y')

In [ ]:
# Convert to GeoDataFrames
df_aos_address_updated = gpd.GeoDataFrame(df_aos_address_updated, crs = 4326, geometry = gpd.points_from_xy(df_aos_address_updated['lon_masked'], df_aos_address_updated['lat_masked'])).to_crs(2056)
df_multiple_address_aos_updated = gpd.GeoDataFrame(df_multiple_address_aos_updated, crs = 4326, geometry = gpd.points_from_xy(df_multiple_address_aos_updated['lon_masked'], df_multiple_address_aos_updated['lat_masked'])).to_crs(2056)
df_multiple_address_lca_updated = gpd.GeoDataFrame(df_multiple_address_lca_updated, crs = 4326, geometry = gpd.points_from_xy(df_multiple_address_lca_updated['lon_masked'], df_multiple_address_lca_updated['lat_masked'])).to_crs(2056)

### Unique address AOS

In [ ]:
# From previous code now deleted, we saw that there is one ID_LAMAL for which there is two address_id
df_aos_address_updated[df_aos_address_updated.duplicated]

In [ ]:
df_aos_address_updated[df_aos_address_updated.duplicated(subset = ['ID_LAMAL'])]

In [ ]:
# We drop the duplicate, the one we choose is unimportant
df_aos_address_updated = df_aos_address_updated.drop_duplicates(['ID_LAMAL'])

### Multiple address AOS

In [ ]:
df_test_multi_aos = df_multiple_address_aos_updated.copy()

In [ ]:
# Simply, if any row has the same ID_LAMAL, year, address_id, lon and lat...then it's just a duplicated row, not multiple addresses
df_test_multi_aos_nouselessdupli = df_test_multi_aos.sort_values(['NOANNEE','MIN_of_Date_adress']).drop_duplicates(subset = ['ID_LAMAL','address_id','lon_masked','lat_masked'], keep = 'first')

In [ ]:
df_test_multi_aos_nouselessdupli.shape[0]

In [ ]:
# Let's check we still have the same number of ID_LAMAL
df_test_multi_aos_nouselessdupli.ID_LAMAL.nunique()

All good

In [ ]:
# List of ID_LAMAL having multiple address_id
dupli_aos = df_test_multi_aos_nouselessdupli[df_test_multi_aos_nouselessdupli.ID_LAMAL.duplicated()].ID_LAMAL.unique()

In [ ]:
# Actually, there are plenty of these that don't have multiple addresses (great data processing on GM's part...)
# This should actually be together with df_aos_address
df_test_multi_notmultipleaddress = df_test_multi_aos_nouselessdupli[df_test_multi_aos_nouselessdupli.ID_LAMAL.isin(dupli_aos) == False].sort_values(['ID_LAMAL','NOANNEE'])
new_df_multiple_address_aos = pd.concat([df_aos_address_updated, df_test_multi_notmultipleaddress[['ID_LAMAL','address_id','lon_masked','lat_masked']]])
new_df_multiple_address_aos['NOANNEE'] = 2017
new_df_multiple_address_aos['MIN_of_Date_adress'] = '10JAN17'

In [ ]:
# These are the ones that actually have real multiple addresses
df_test_multi_aos_realmultiaddress = df_test_multi_aos_nouselessdupli[df_test_multi_aos_nouselessdupli.ID_LAMAL.isin(dupli_aos)].sort_values(['ID_LAMAL','NOANNEE'])

In [ ]:
# Due to geomasking we have cases where the coordinates are almost exactly the same, these can be identified by distance calculation and collasped into one
same_date_different_address = df_test_multi_aos_realmultiaddress[df_test_multi_aos_realmultiaddress.duplicated(subset = ['ID_LAMAL','date'])].ID_LAMAL.unique()
df_same_date_different_address = df_test_multi_aos_realmultiaddress[df_test_multi_aos_realmultiaddress.ID_LAMAL.isin(same_date_different_address)]

In [ ]:
# Then we have the ones with multiples addresses at different dates aka no problem !
df_test_multi_aos_realmultiaddress_noproblem = df_test_multi_aos_realmultiaddress[df_test_multi_aos_realmultiaddress.ID_LAMAL.isin(same_date_different_address) == False]

In [ ]:
# Calculating distance between addresses of each individual
df_same_date_different_address["distance"] = df_same_date_different_address.groupby("ID_LAMAL", observed = True)["geometry"].apply(lambda x: x.distance(x.shift()))

In [ ]:
# Checking the few ones with addresses located at less than or equal to 500m
# For these, I could just pick one address per date, assuming that it won't change much since they are very close
same_date_different_address_closedistance = df_same_date_different_address[df_same_date_different_address['distance'] <= 500].ID_LAMAL.unique()
same_date_different_address_fardistance = df_same_date_different_address[df_same_date_different_address['distance'] > 500].ID_LAMAL.unique()

In [ ]:
df_same_date_different_address_closedistance  = df_same_date_different_address[df_same_date_different_address.ID_LAMAL.isin(same_date_different_address_closedistance)]
df_same_date_different_address_fardistance  = df_same_date_different_address[df_same_date_different_address.ID_LAMAL.isin(same_date_different_address_fardistance)]

In [ ]:
# Since we only check for a distance threshold some addresses may have one under 500 and another one above, if there is even one under, we don't want it in the far dataset, so we remove it
df_same_date_different_address_fardistance = df_same_date_different_address_fardistance[df_same_date_different_address_fardistance.ID_LAMAL.isin(same_date_different_address_closedistance) == False]

In [ ]:
# Picking one address per date for close distance duplicates
df_same_date_different_address_closedistance_nodupli = df_same_date_different_address_closedistance.drop_duplicates(['MIN_of_Date_adress','ID_LAMAL'], keep = 'first')

In [ ]:
# Picking one address per date for far distance duplicates, we order by ID, date and distance (NaN being last), and keep the last
df_same_date_different_address_fardistance_nodupli = df_same_date_different_address_fardistance.sort_values(['ID_LAMAL','date','distance']).drop_duplicates(['ID_LAMAL','date'], keep = 'last')

### Putting AOS back together

In [ ]:
# 1 Real unique addresses
print(df_aos_address_updated.ID_LAMAL.nunique())
# 2 "Fake" multiple addresses
print(df_test_multi_notmultipleaddress.ID_LAMAL.nunique())
# 3 Non-problematic multiple addresses
print(df_test_multi_aos_realmultiaddress_noproblem.ID_LAMAL.nunique())
# 4 Problematic - Same date different addresses - Close distance duplicates
print(df_same_date_different_address_closedistance_nodupli.ID_LAMAL.nunique())
# 5 Problematic - Same date different addresses - Far distance duplicates
print(df_same_date_different_address_fardistance_nodupli.ID_LAMAL.nunique())

In [ ]:
# Concatenate the different datasets to create a final AOS address df
df_aos_addresses_final = pd.concat([df_aos_address_updated,
          df_test_multi_notmultipleaddress,
          df_test_multi_aos_realmultiaddress_noproblem,
          df_same_date_different_address_closedistance_nodupli,
          df_same_date_different_address_fardistance_nodupli])

In [ ]:
# Let's check if there is any unwanted duplicates
print('Number of duplicates: ', df_aos_addresses_final[df_aos_addresses_final.duplicated(subset = ['ID_LAMAL','date'])].shape[0])

In [ ]:
# Let's check the final number of ID_LAMAL
print('Number of ID_LAMAL: ', df_aos_addresses_final.ID_LAMAL.nunique())

In [ ]:
df_aos_addresses_final['uuid'] = df_aos_addresses_final['ID_LAMAL'].map(dict_lamal_to_uuid)

In [ ]:
df_remaining_multiple_aos_updated.ID_Lamal.nunique()

## LCA addresses

In [ ]:
df_aos_addresses_final['ID_LCA'] = df_aos_addresses_final['ID_LAMAL'].map(df_paires_lamal_lca.set_index('id_lamal')['id_lca'].to_dict())

## Geographical units

### Lakes

In [ ]:
lakes = gpd.read_file("/Users/david/Dropbox/PhD/GitHub/COVID19/input/g2s15.shp")

### Country boundaries

In [ ]:
country_geo = gpd.read_file("/Users/david/Dropbox/PhD/GitHub/COVID19/input/g2l15.shp")
with fiona.open("/Users/david/Dropbox/PhD/GitHub/COVID19/input/g2l15.shp", "r") as shapefile:
    country_geo_fiona = [feature["geometry"] for feature in shapefile]

In [ ]:
ch_extent = np.asarray(country_geo.bounds)[0][[0,2,1,3]]

### Relief

In [ ]:
# Open the file:
relief_raster = rasterio.open('/Users/david/Dropbox/PhD/GitHub/COVID19/input/02-relief-ascii.asc')

In [ ]:
out_image, out_transform = rasterio.mask.mask(relief_raster, country_geo_fiona, crop=True, filled=False)
out_meta = relief_raster.meta

### Cantons

In [ ]:
cantons_ch = gpd.read_file(data_folder/'raw/Linkage/swissBOUNDARIES3D_1_3_TLM_KANTONSGEBIET.shp')
cantons_ch = cantons_ch.to_crs(21781)

## Demography

In [ ]:
statpop = pd.read_csv(data_folder/'raw/OFS/STATPOP/ag-b-00.03-vz2020statpop/STATPOP2020.csv',sep = ';')
statpop_ha = statpop.copy()
geometry = [Point(xy) for xy in zip(statpop['E_KOORD'], statpop['N_KOORD'])]
statpop_point = gpd.GeoDataFrame(statpop, crs=2056, geometry=geometry)   

In [ ]:
geometry = [Polygon(zip([xy[0],xy[0],xy[0]+100,xy[0]+100],[xy[1],xy[1]+100,xy[1]+100,xy[1]])) for xy in zip(statpop_ha.E_KOORD, statpop_ha.N_KOORD)]
statpop_ha = gpd.GeoDataFrame(statpop_ha, crs=2056, geometry=geometry)      

## Environment

In [ ]:
gdf_lst = pd.read_pickle('../data/processed/gdf_lst.pkl')
gdf_lst = gpd.GeoDataFrame(gdf_lst, geometry = gdf_lst['geometry'])
gdf_ndvi = pd.read_pickle('../data/processed/gdf_ndvi.pkl')
gdf_ndvi = gpd.GeoDataFrame(gdf_ndvi, geometry = gdf_ndvi['geometry'])
pm10 = pd.read_pickle('../data/processed/pm10_2020.pkl')
# pm10 = gpd.GeoDataFrame(pm10, geometry = pm10['geometry'])
pm25 = pd.read_pickle('../data/processed/pm25_2020.pkl')
# pm25 = gpd.GeoDataFrame(pm25, geometry = pm25['geometry'])
no2 = pd.read_pickle('../data/processed/no2_2020.pkl')
# no2_2020 = gpd.GeoDataFrame(no2_2020, geometry = no2_2020['geometry'])
ns_car_day = pd.read_pickle('../data/processed/ns_car_day.pkl')
# ns_car_day = gpd.GeoDataFrame(ns_car_day, geometry = ns_car_day['geometry'])
ns_car_night = pd.read_pickle('../data/processed/ns_car_night.pkl')
# ns_car_night = gpd.GeoDataFrame(ns_car_night, geometry = ns_car_night['geometry'])

In [ ]:
pm10 = pd.DataFrame(pm10).rename(columns = {'mean':'mean_pm10','median':'median_pm10'})
pm25 = pd.DataFrame(pm25).rename(columns = {'mean':'mean_pm25','median':'median_pm25'})
no2 = pd.DataFrame(no2).rename(columns = {'mean':'mean_no2','median':'median_no2'})
ns_car_day = pd.DataFrame(ns_car_day).rename(columns = {'mean':'mean_carday','median':'median_carday'})
ns_car_night = pd.DataFrame(ns_car_night).rename(columns = {'mean':'mean_carnight','median':'median_carnight'})

In [ ]:
pollution_df = pd.concat([pm10, pm25, no2, ns_car_day, ns_car_night], axis = 1)

In [ ]:
def pct_replacement(column):
    conds = [column > np.percentile(column, 99.9)]
    choices = [np.percentile(column, 50)]
    return np.select(conds,choices,column)

In [ ]:
pollution_df = pollution_df.apply(lambda x: pct_replacement(x))
pollution_df = pd.concat([statpop_ha[['RELI','geometry']], pollution_df], axis = 1)
pollution_df = gpd.GeoDataFrame(pollution_df, crs = 2056, geometry = pollution_df['geometry'])

In [ ]:
# Converting polygon geometries to point using centroids of polygons for NDVI
gdf_ndvi['geometry_pt'] = gdf_ndvi['geometry'].centroid
gdf_ndvi = gdf_ndvi.set_geometry("geometry_pt")

# Converting polygon geometries to point using centroids of polygons for LST
gdf_lst['geometry_pt'] = gdf_lst['geometry'].centroid
gdf_lst = gdf_lst.set_geometry("geometry_pt")

In [ ]:
classy_map_ch(gdf_ndvi, 'mean_ndvi','RdYlGn', 'Indice de végétation', False)

In [ ]:
classy_map_ch(gdf_lst, 'mean_lst','magma', 'Température de surface (ºC)', False)

In [ ]:
# Converting polygon geometries to point using centroids of polygons for pollution variables
pollution_df['geometry_pt'] = pollution_df['geometry'].centroid
pollution_df = pollution_df.set_geometry("geometry_pt")

In [ ]:
classy_map_ch(pollution_df, 'mean_pm10','magma', 'Pollution atmosphérique (PM10)', False)

In [ ]:
classy_map_ch(pollution_df, 'mean_pm25','magma', 'Pollution atmosphérique (PM25)', False)

In [ ]:
classy_map_ch(pollution_df, 'mean_no2','magma', 'Pollution atmosphérique (NO2)', False)

In [ ]:
classy_map_ch(pollution_df, 'mean_carday','magma', 'Pollution sonore (dB)', False)

## Socioeconomic index

In [ ]:
df_swiss_sep1 = optimize_df(gpd.read_file(data_folder/'raw'/'Swiss-SEP'/'SNC_Swiss-SEP1'/'SHP'/'ssep_user_geo.shp', driver = 'Shapefile'))
df_swiss_sep2 = optimize_df(gpd.read_file(data_folder/'raw'/'Swiss-SEP'/'SNC_Swiss-SEP2'/'SHP'/'ssep2_user_geo.shp', driver = 'Shapefile'))

In [ ]:
df_swiss_2023 = optimize_df(pd.read_csv(data_folder/'raw'/'Swiss-SEP'/'SNC_Swiss-SEP-2023'/'ssep_open.csv'))

In [ ]:
df_swiss_2023 = gpd.GeoDataFrame(df_swiss_2023, crs = 2056, geometry=gpd.points_from_xy(df_swiss_2023.geox, df_swiss_2023.geoy))

In [ ]:
cantons_ch = cantons_ch.to_crs(2056)

In [ ]:
cantons_ch[cantons_ch.NAME == 'Genève']

In [ ]:
df_swiss_sep_ge = df_swiss_2023[df_swiss_2023.within(cantons_ch.loc[20].geometry)]

In [ ]:
df_swiss_sep_ge[['ssep3','geometry']].explore('ssep3', cmap = 'RdYlGn', marker_kwds = {'radius':1})

In [ ]:
ax = df_swiss_sep_ge.plot('ssep3', cmap = 'RdYlGn', markersize = 0.5, figsize = (15,15), linewidth=0, legend = True, legend_kwds={'shrink':0.5})
ax.set_axis_off()
plt.savefig(result_folder/'Maps features'/'Indice de statut socioéconomique - GE.png', dpi=1200, bbox_inches='tight')

In [ ]:
ax = df_swiss_sep_ge.plot('ssep3_d', cmap = 'RdYlGn', vmin = 0, vmax = 10, markersize = 0.5, figsize = (15,15), linewidth=0, legend = True, legend_kwds={'shrink':0.5})
ax.set_axis_off()
plt.savefig(result_folder/'Maps features'/'Indice de statut socioéconomique (Déciles) - GE.png', dpi=1200, bbox_inches='tight')

In [ ]:
classy_map_ch(df_swiss_sep2, 'ssep2_d','RdYlGn', 'Indice de statut socioéconomique (Déciles)', False)

In [ ]:
classy_map_ch(df_swiss_sep2, 'ssep2','RdYlGn', 'Indice de statut socioéconomique', False)

In [ ]:
classy_map_ch(df_swiss_2023, 'ssep3_d','RdYlGn', 'Indice de statut socioéconomique 2023 (Déciles)', False)

## Accessibilité

In [ ]:
df_access_ofs = pd.read_csv(data_folder/'raw/OFS/Accessibility/ag-b-00.03-2018spop-csv.csv', sep = ';')

In [ ]:
df_access_ofs = gpd.GeoDataFrame(df_access_ofs, crs = 2056, geometry = gpd.points_from_xy(df_access_ofs.E_COORD, df_access_ofs.N_COORD))

In [ ]:
classy_map_ch(df_access_ofs, 'D_MEDIC', 'RdYlGn', 'Accès aux cabinets médicaux et centres ambulatoires')

In [ ]:
classy_map_ch(df_access_ofs, 'D_MEDIC_B', 'RdYlGn', 'Accès à la médecine de premier recours')

In [ ]:
classy_map_ch(df_access_ofs, 'D_MEDIC_S', 'RdYlGn', 'Accès à la médecine spécialisée')

### Add sociodemographic and environmental features

Accessibility data

In [ ]:
geometry = [Polygon(zip([xy[0],xy[0],xy[0]+100,xy[0]+100],[xy[1],xy[1]+100,xy[1]+100,xy[1]])) for xy in zip(df_access_ofs.E_COORD, df_access_ofs.N_COORD)]

df_access_ofs = df_access_ofs.set_geometry(geometry)

In [ ]:
df_addresses_w_access = gpd.sjoin_nearest(df_aos_addresses_final, df_access_ofs.drop(['RELI','E_COORD','N_COORD','YEAR','POP_TOTAL'], axis = 1), how = 'left', distance_col = 'distance_join_access').drop('index_right', axis = 1)

Pollution data

In [ ]:
# geometry = [Polygon(zip([xy[0],xy[0],xy[0]+100,xy[0]+100],[xy[1],xy[1]+100,xy[1]+100,xy[1]])) for xy in zip(pollution_df.geometry_pt.x, pollution_df.geometry_pt.y)]
pollution_df = pollution_df.set_geometry('geometry')

In [ ]:
df_addresses_w_access_pollution = gpd.sjoin_nearest(df_addresses_w_access, pollution_df.drop(['RELI','geometry_pt'], axis = 1), how = 'left', distance_col = 'distance_join_pollution').drop('index_right', axis = 1)

NDVI & LST

In [ ]:
gdf_ndvi_lst = pd.concat([gdf_ndvi.drop(['geometry_pt','geometry'], axis = 1), gdf_lst], axis = 1)

In [ ]:
gdf_ndvi_lst = gdf_ndvi_lst.set_geometry('geometry')

In [ ]:
df_addresses_w_access_pollution_ndvi_lst = gpd.sjoin_nearest(df_addresses_w_access_pollution, gdf_ndvi_lst.drop(['geometry_pt'], axis = 1), how = 'left', distance_col = 'distance_join_ndvi_lst').drop('index_right', axis = 1)

Socioeconomic index

In [ ]:
geometry = [Polygon(zip([xy[0],xy[0],xy[0]+100,xy[0]+100],[xy[1],xy[1]+100,xy[1]+100,xy[1]])) for xy in zip(df_swiss_2023.geox, df_swiss_2023.geoy)]
df_swiss_2023 = df_swiss_2023.set_geometry(geometry)
df_swiss_2023.crs = 2056

In [ ]:
df_aos_addresses_final_sep = gpd.sjoin_nearest(df_aos_addresses_final, df_swiss_2023, how = 'left')

In [ ]:
df_aos_addresses_final_sep = df_aos_addresses_final_sep.drop_duplicates(subset = ['ID_LAMAL','address_id','date'])

In [ ]:
df_addresses_w_access_pollution_ndvi_lst_sep = pd.merge(df_addresses_w_access_pollution_ndvi_lst,df_aos_addresses_final_sep[['uuid','address_id','date','ssep2','ssep2_d','ssep2_t','ssep2_q','ssep3','ssep3_d','ssep3_t','ssep3_q']], on = ['uuid','address_id','date'], how = 'left')

In [ ]:
df_addresses_w_access_pollution_ndvi_lst_sep.to_parquet(data_folder/'/processed/df_addresses_with_socio_env.parquet.gzip', compression = 'gzip')

In [ ]:
df_addresses_w_access_pollution_ndvi_lst_sep = gpd.GeoDataFrame(df_addresses_w_access_pollution_ndvi_lst_sep, crs = 4326, geometry=gpd.points_from_xy(df_addresses_w_access_pollution_ndvi_lst_sep.lon_masked, df_addresses_w_access_pollution_ndvi_lst_sep.lat_masked))
df_addresses_w_access_pollution_ndvi_lst_sep_uniques = df_addresses_w_access_pollution_ndvi_lst_sep[df_addresses_w_access_pollution_ndvi_lst_sep.doubl.isnull()]

In [ ]:
# generate list of years
years = [2017, 2018, 2019, 2020, 2021]

# repeat rows for each year
df_addresses_w_access_pollution_ndvi_lst_sep_uniques = df_addresses_w_access_pollution_ndvi_lst_sep_uniques.reindex(df_addresses_w_access_pollution_ndvi_lst_sep_uniques.index.repeat(len(years)))
df_addresses_w_access_pollution_ndvi_lst_sep_uniques['NOANNEE'] = years * (len(df_addresses_w_access_pollution_ndvi_lst_sep_uniques.index)//len(years))

In [ ]:
df_full_address = pd.concat([df_addresses_w_access_pollution_ndvi_lst_sep_uniques, df_addresses_w_access_pollution_ndvi_lst_sep[df_addresses_w_access_pollution_ndvi_lst_sep.doubl.isnull() == False]]).reset_index(drop=True)
df_full_address = df_full_address.sort_values(['uuid','date']).drop_duplicates(subset = ['uuid','NOANNEE'], keep = 'first')

In [ ]:
df_full_address.to_parquet(data_folder/'processed/df_full_address.parquet.gzip', compression = 'gzip')

In [ ]:
df_franchise = df_couverture_aos[['ID_LAMAL','MTFRANCHISECOUV']].drop_duplicates()

In [ ]:
df_franchise = pd.merge(df_franchise, df_address_full[['ID_LAMAL','geometry']].drop_duplicates(), on = 'ID_LAMAL')
df_franchise = df_franchise.set_geometry('geometry')
df_franchise = df_franchise.to_crs(2056)

In [ ]:
# wnn16 = lps.weights.KNN(cKDTree(get_points_array(df_franchise.geometry)),50)
# getiswnn16 = pyspace.compute_getis(df_franchise,'MTFRANCHISECOUV',wnn16, 999, p_001 = False)
# fdr_pvalue = fdr(getiswnn16.p_sim, 0.05)

In [ ]:
df_age = df_couverture_aos[['ID_LAMAL','NBAGE']].groupby('ID_LAMAL', observed = True)['NBAGE'].max()
df_age = pd.merge(df_age, df_address_full[['ID_LAMAL','geometry']].drop_duplicates(), on = 'ID_LAMAL')
df_age = df_age.set_geometry('geometry')
df_age = df_age.to_crs(2056)

In [ ]:
df_age = df_age.sample(120000)

In [ ]:
df_age['E'] = df_age['geometry'].x

In [ ]:
# wnn16 = lps.weights.KNN(cKDTree(get_points_array(df_age.geometry)),50)

In [ ]:
# from esda.getisord import G_Local

# g = G_Local(df_age['NBAGE'], wnn16, star = True, permutations = 999)


In [ ]:
# getiswnn16 = pyspace.compute_getis(df_age,'NBAGE',wnn16, 999, p_001 = False)
# fdr_pvalue = fdr(getiswnn16.p_sim, 0.05)

In [ ]:
# df_age['NBAGE_G_cl_fdr'] = df_age['NBAGE_G_cl']  
# df_age.loc[df_age['NBAGE_G_psim'] >= fdr_pvalue, 'NBAGE_G_cl_fdr'] = 'Not significant'
# fig, ax = pyspace.plotGetisMap(df_age,'NBAGE_G_cl',p_001 = False, commune_name = False)